Recordar consultar el [Diccionario de datos](./Datasets/Google%20Maps/Diccionario%20de%20datos.docx)


# Importaciones

In [6]:
import pandas as pd

# Llamado de datos a dataframes

In [7]:
df_maps_restaurantes = pd.read_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews = pd.read_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes = pd.read_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin = pd.read_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips = pd.read_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews = pd.read_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users = pd.read_parquet(r'Generated\Yelp\users_extracted.parquet')

In [8]:
df_maps_restaurantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212014 entries, 0 to 212013
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   name                    212014 non-null  object 
 1   address                 210956 non-null  object 
 2   gmap_id                 212014 non-null  object 
 3   description             72392 non-null   object 
 4   latitude                212014 non-null  float64
 5   longitude               212014 non-null  float64
 6   category                212014 non-null  object 
 7   avg_rating              212014 non-null  float64
 8   num_of_reviews          212014 non-null  int64  
 9   price                   100000 non-null  object 
 10  hours                   0 non-null       object 
 11  state                   190751 non-null  object 
 12  relative_results        170957 non-null  object 
 13  url                     212014 non-null  object 
 14  MISC.Accessibility  

In [9]:
df_maps_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393452 entries, 0 to 2393451
Data columns (total 41 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 object 
 1   name_x                  object 
 2   time                    int64  
 3   rating                  int64  
 4   text                    object 
 5   pics                    object 
 6   gmap_id                 object 
 7   anio                    int64  
 8   estado                  object 
 9   name_y                  object 
 10  address                 object 
 11  description             object 
 12  latitude                float64
 13  longitude               float64
 14  category                object 
 15  avg_rating              float64
 16  num_of_reviews          int64  
 17  price                   object 
 18  hours                   object 
 19  state                   object 
 20  relative_results        object 
 21  url                     object 

In [10]:
df_yelp_restaurantes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22685 entries, 3.0 to 150339.0
Data columns (total 58 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   business_id                            22685 non-null  object 
 1   name                                   22685 non-null  object 
 2   address                                22685 non-null  object 
 3   city                                   22685 non-null  object 
 4   state                                  22685 non-null  object 
 5   postal_code                            22685 non-null  object 
 6   latitude                               22685 non-null  float64
 7   longitude                              22685 non-null  float64
 8   stars                                  22685 non-null  float64
 9   review_count                           22685 non-null  int64  
 10  is_open                                22685 non-null  int64  
 11  ca

In [11]:
df_yelp_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3330 entries, 0 to 3329
Data columns (total 57 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   business_id                            3330 non-null   object 
 1   name                                   3330 non-null   object 
 2   address                                3330 non-null   object 
 3   city                                   3330 non-null   object 
 4   state                                  3330 non-null   object 
 5   postal_code                            3330 non-null   object 
 6   latitude                               3330 non-null   float64
 7   longitude                              3330 non-null   float64
 8   stars                                  3330 non-null   float64
 9   review_count                           3330 non-null   int64  
 10  is_open                                3330 non-null   int64  
 11  cate

In [12]:
df_yelp_tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69346 entries, 0 to 69345
Data columns (total 62 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   user_id                                69346 non-null  object 
 1   business_id                            69346 non-null  object 
 2   text                                   69346 non-null  object 
 3   date                                   69346 non-null  object 
 4   compliment_count                       69346 non-null  int64  
 5   name                                   69346 non-null  object 
 6   address                                69346 non-null  object 
 7   city                                   69346 non-null  object 
 8   state                                  69346 non-null  object 
 9   postal_code                            69346 non-null  object 
 10  latitude                               69346 non-null  float64
 11  lo

In [13]:
df_yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558655 entries, 0 to 558654
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_id    558655 non-null  object
 1   user_id      558655 non-null  object
 2   business_id  558655 non-null  object
 3   stars        558655 non-null  int8  
 4   funny        558655 non-null  int8  
 5   cool         558655 non-null  int8  
 6   text         558655 non-null  object
 7   date         558655 non-null  object
dtypes: int8(3), object(5)
memory usage: 22.9+ MB


In [14]:
df_yelp_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99015 entries, 0 to 99014
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             99015 non-null  object 
 1   name                99015 non-null  object 
 2   review_count        99015 non-null  int64  
 3   yelping_since       99015 non-null  object 
 4   useful              99015 non-null  int64  
 5   funny               99015 non-null  int64  
 6   cool                99015 non-null  int64  
 7   elite               99015 non-null  object 
 8   friends             99015 non-null  object 
 9   fans                99015 non-null  int64  
 10  average_stars       99015 non-null  float64
 11  compliment_hot      99015 non-null  int64  
 12  compliment_more     99015 non-null  int64  
 13  compliment_profile  99015 non-null  int64  
 14  compliment_cute     99015 non-null  int64  
 15  compliment_list     99015 non-null  int64  
 16  comp